In [14]:
# 20-dimensional Optimization of Ackley Function using BOTorch

import torch

from gpytorch import kernels, means, likelihoods
from gpytorch.mlls import ExactMarginalLogLikelihood

from pyDOE3 import lhs

import numpy as np
import matplotlib.pyplot as plt

#from scipy.interpolate import griddata

from botorch.models.transforms import Standardize, Normalize
#from botorch.utils.transforms import normalize, unnormalize
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_mll
from botorch.acquisition.logei import qLogNoisyExpectedImprovement
from botorch.models.transforms.outcome import Standardize
from botorch.optim import optimize_acqf

from botorch.test_functions import Ackley

In [41]:
seed =42
torch.manual_seed(seed)

noise_std = 0.5

initial_sample = 24

nu = 2.5

dim = 10

bounds = torch.stack([torch.zeros(dim), torch.ones(dim)])

# standard Latin hypercube using maximin criterion (converts to tensor)
lhs_design = torch.tensor(
            lhs(n = dim, samples = initial_sample, criterion = 'maximin', random_state=seed), 
        dtype=torch.double
        )

#print(lhs_design, lhs_design.shape)

In [23]:
# Gaussian Process Model

# Only for further GP Model development

# Setup (Assume train_X, train_Y, bounds, and objective_function are defined)
# ===========================



# GP Model definition
class GPModel(SingleTaskGP):
    def __init__(self, train_X, train_Y, fixed_noise=False, noise_level=noise_level,
                 lengthscale_prior=None, outputscale_prior=None,
                 lengthscale_constraint = None, outputscale_constraint=None):

        if fixed_noise:
            print(f"Training with FIXED noise: {noise_level*100:.1f}% of mean output.")
            noise_variance = (noise_level * train_Y.mean()).pow(2)
            train_Yvar = torch.full_like(train_Y, noise_variance)
            likelihood = None
            super().__init__(
                train_X, train_Y, train_Yvar=train_Yvar, likelihood=likelihood,
                outcome_transform=Standardize(m=1),
                input_transform=Normalize(d=dim)
            )
        else:
            print("Training with LEARNABLE noise (Gaussian Likelihood).")
            likelihood = likelihoods.GaussianLikelihood()
            super().__init__(
                train_X, train_Y, likelihood=likelihood,
                outcome_transform=Standardize(m=1),
                input_transform=Normalize(d=dim)
            )
            '''
            lower_noise = lower_noise_bound**2  # lower noise bound
            upper_noise = upper_noise_bound**2  # upper noise bound

            # Add a **prior** (softly nudges during training)
            self.likelihood.noise_covar.register_prior(
                "noise_prior",
                SmoothedBoxPrior(lower_noise, upper_noise),
                "raw_noise"
            )

            # Add a **constraint** (hard bounding box)
            self.likelihood.noise_covar.register_constraint(
                "raw_noise",
                Interval(lower_noise, upper_noise)
            )
            '''

        self.mean_module = means.ConstantMean()

        matern_kernel = kernels.MaternKernel(
            nu=nu,
            ard_num_dims=dim,
            lengthscale_prior=lengthscale_prior,
            lengthscale_constraint=lengthscale_constraint,
        )

        self.covar_module = kernels.ScaleKernel(
            base_kernel=matern_kernel,
            outputscale_prior=outputscale_prior,
            outputscale_constraint=outputscale_constraint,
        )
        #self.likelihood=likelihood  # I added this to fix for fixed, but it might be redundant

        
# Training function
def train_GP_model(train_X, train_Y, fixed_noise=False, noise_level=noise_level,
                   lengthscale_prior=None, outputscale_prior=None,
                   lengthscale_constraint = None, outputscale_constraint=None): 
    model = GPModel(
        train_X, train_Y,
        fixed_noise=fixed_noise,
        noise_level=noise_level,
        lengthscale_prior=lengthscale_prior,
        outputscale_prior=outputscale_prior,
        lengthscale_constraint = lengthscale_constraint, 
        outputscale_constraint=outputscale_constraint
    )
    mll = ExactMarginalLogLikelihood(model.likelihood, model)
    fit_gpytorch_mll(mll)
    return model, mll


In [ ]:
n_iterations = 200
num_restarts =50
raw_samples =50
batch_size = 1

# the multidimensional function (where best answer is 0, where all X = 0)
ackley = Ackley(dim=dim, noise_std = noise_std, negate=True) #negate = True to maximize the function

train_X = lhs_design
train_Y = ackley(train_X).unsqueeze(-1)

#print(train_Y, train_Y.shape)

for iterations in range (n_iterations):
    print(f"\n ===Iteration {iterations+1}===")
    # Train the GP model
    model, mll = train_GP_model(
        train_X, train_Y,
        fixed_noise=False,
        #noise_level=noise_level,
        lengthscale_prior=None,
        outputscale_prior=None,
        lengthscale_constraint = None, 
        outputscale_constraint=None
    )

    model.eval()

    acq_func = qLogNoisyExpectedImprovement(model = model,
                                            X_baseline=train_X,
                                            prune_baseline = True, # default
                                            cache_root = True, # default True (uses Cholesky decomposition)
                                            )
    
    #Optimize acq func
    candidate, _ = optimize_acqf(
        acq_function=acq_func,
        q=batch_size,
        bounds=bounds,
        num_restarts=num_restarts,
        raw_samples=raw_samples,
    )

    new_y = torch.stack([ackley(c.unsqueeze(0)) for c in candidate])

    train_X = torch.cat([train_X, candidate], dim=0)
    train_Y = torch.cat([train_Y, new_y], dim=0)

    best_idx = torch.argmax(train_Y.squeeze(-1))
    best_X = train_X[best_idx]
    best_Y = train_Y[best_idx].item()
    print(f"Best Y: {best_Y:.4f} at X: {best_X}")
    


 ===Iteration 1===
Training with LEARNABLE noise (Gaussian Likelihood).
Best Y: -2.5416 at X: tensor([0.2470, 0.2308, 0.6556, 0.0511, 0.4228, 0.2537, 0.3460, 0.6021, 0.7282,
        0.0514], dtype=torch.float64)

 ===Iteration 2===
Training with LEARNABLE noise (Gaussian Likelihood).
Best Y: -2.5416 at X: tensor([0.2470, 0.2308, 0.6556, 0.0511, 0.4228, 0.2537, 0.3460, 0.6021, 0.7282,
        0.0514], dtype=torch.float64)

 ===Iteration 3===
Training with LEARNABLE noise (Gaussian Likelihood).
Best Y: -2.5416 at X: tensor([0.2470, 0.2308, 0.6556, 0.0511, 0.4228, 0.2537, 0.3460, 0.6021, 0.7282,
        0.0514], dtype=torch.float64)

 ===Iteration 4===
Training with LEARNABLE noise (Gaussian Likelihood).
Best Y: -2.5416 at X: tensor([0.2470, 0.2308, 0.6556, 0.0511, 0.4228, 0.2537, 0.3460, 0.6021, 0.7282,
        0.0514], dtype=torch.float64)

 ===Iteration 5===
Training with LEARNABLE noise (Gaussian Likelihood).
Best Y: -2.5416 at X: tensor([0.2470, 0.2308, 0.6556, 0.0511, 0.4228, 0.253

c:\Users\uqkmuroi\gitcode\Torch-Practice\venv\Lib\site-packages\linear_operator\utils\cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  warnings.warn(


Best Y: -0.5294 at X: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=torch.float64)

 ===Iteration 57===
Training with LEARNABLE noise (Gaussian Likelihood).


c:\Users\uqkmuroi\gitcode\Torch-Practice\venv\Lib\site-packages\linear_operator\utils\cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  warnings.warn(


Best Y: -0.5294 at X: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=torch.float64)

 ===Iteration 58===
Training with LEARNABLE noise (Gaussian Likelihood).


...

c:\Users\uqkmuroi\gitcode\Torch-Practice\venv\Lib\site-packages\linear_operator\utils\cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  warnings.warn(


Best Y: 1.4273 at X: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=torch.float64)

 ===Iteration 150===
Training with LEARNABLE noise (Gaussian Likelihood).


c:\Users\uqkmuroi\gitcode\Torch-Practice\venv\Lib\site-packages\linear_operator\utils\cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  warnings.warn(


Best Y: 1.4273 at X: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=torch.float64)

 ===Iteration 151===
Training with LEARNABLE noise (Gaussian Likelihood).


c:\Users\uqkmuroi\gitcode\Torch-Practice\venv\Lib\site-packages\linear_operator\utils\cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  warnings.warn(
